# Window functions (OVER)

Check following sources:

- [SQL window functions](https://mode.com/sql-tutorial/sql-window-functions) tutorial on mode.com.
- [Window functions](https://www.postgresql.org/docs/9.1/tutorial-window.html) article on official postgres documentation.

A window function performs a calculation across a set of table rows that are somehow related to the current row. This is comparable to the type of calculation that can be done with an aggregate function. But unlike regular aggregate functions, use of a window function does not cause rows to become grouped into a single output row — the rows retain their separate identities. Behind the scenes, the window function is able to access more than just the current row of the query result.

In fact, you are doing something like aggregation, but the results are not collapsed by variable aggregation.

Let's consider an example for this page. We have a numeric variable called `some_variable`, on which we will apply various functions. Also, we have a `group` variable that represents the relationship between rows.

In [1]:
%%bash
docker run -d --rm\
    --name window_functions_example\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5
docker exec -i window_functions_example psql -U postgres -d postgres

CREATE TABLE example_table (
    some_value INT NOT NULL,
    "group" TEXT NOT NULL
);
INSERT INTO example_table (some_value, "group") VALUES
(2, 'A'),
(1, 'A'),
(9, 'A'),
(4, 'A'),
(8, 'A'),
(0, 'B'),
(0, 'A'),
(5, 'A'),
(8, 'B'),
(5, 'A');

SELECT * FROM example_table;

CREATE TABLE
INSERT 0 10
 some_value | group 
------------+-------
          2 | A
          1 | A
          9 | A
          4 | A
          8 | A
          0 | B
          0 | A
          5 | A
          8 | B
          5 | A
(10 rows)



**Note** Don't forget to stop the container.

In [ ]:
!docker stop window_functions_example

## Syntax

If you are using a window function in the `SELECT` block of your query, you can declare a new output column using the following syntax:

```sql<function> OVER (<relationship>) AS <result name>.```

This allows you to compute the result of the window function and assign it to a new column with the specified `<result name>`.

In [17]:
%%bash
docker exec -i window_functions_example psql -U postgres -d postgres

SELECT 
    some_value, "group",
    SUM(some_value) OVER (PARTITION BY "group") AS sv
FROM example_table;

 some_value | group | sv 
------------+-------+----
          2 | A     | 34
          1 | A     | 34
          9 | A     | 34
          4 | A     | 34
          8 | A     | 34
          5 | A     | 34
          5 | A     | 34
          0 | A     | 34
          0 | B     |  8
          8 | B     |  8
(10 rows)

